# Transfer Learning  with TensorFlow Part 2: Fine-Tuning

In [1]:
!nvidia-smi

Thu Dec 28 10:17:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 41%   63C    P0    46W / 180W |   2002MiB /  8192MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Creating helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-12-28 10:21:50--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2023-12-28 10:21:51 (14.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Let's get some data

This time we're going to see how we can use the pretrained models within `tf.keras.application`

In [9]:
# Get 10% of training data of 10 classes of Food101
!wget "https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip"

--2023-12-28 10:37:03--  http://../https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving .. (..)... failed: Name or service not known.
wget: unable to resolve host address ‘..’


In [6]:
import zipfile

filename = "../10_food_classes_10_percent.zip"
zip_ref = zipfile.ZipFile(filename, "r")
zip_ref.extractall()
zip_ref.close()

In [8]:
import os

dir_path = "./10_food_classes_10_percent"
for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in './10_food_classes_10_percent'.
There are 10 directories and 0 images in './10_food_classes_10_percent/test'.
There are 0 directories and 250 images in './10_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in './10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in './10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in './10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in './10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in './10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in './10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in './10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in './10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in './10

In [10]:
# Create training and test directory paths
train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"

In [11]:
import tensorflow as tf

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                           image_size=IMG_SIZE,
                                                                           label_mode="categorical",
                                                                           batch_size=BATCH_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                               image_size=IMG_SIZE,
                                                               label_mode="categorical",
                                                               batch_size=BATCH_SIZE)

2023-12-28 11:04:33.432424: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-28 11:04:33.432486: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-28 11:04:33.543752: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-28 11:04:33.730740: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-28 11:04:35.367285: W tensorflow/compiler/tf2

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


2023-12-28 11:04:37.757075: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-28 11:04:38.030271: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [12]:
train_data_10_percent

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [14]:
# Check out the class names of our dataset
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [15]:
# See an example of a batch of data
for images, labels in train_data_10_percent.take(1):
    print(images, labels)

tf.Tensor(
[[[[3.80000000e+01 0.00000000e+00 0.00000000e+00]
   [3.80000000e+01 0.00000000e+00 0.00000000e+00]
   [3.65714264e+01 0.00000000e+00 0.00000000e+00]
   ...
   [1.20734795e+02 1.04576653e+02 9.42960358e+01]
   [1.24954109e+02 1.11311256e+02 1.02311256e+02]
   [1.31801147e+02 1.17515434e+02 1.10158287e+02]]

  [[3.80000000e+01 0.00000000e+00 0.00000000e+00]
   [3.80000000e+01 0.00000000e+00 0.00000000e+00]
   [3.65714264e+01 0.00000000e+00 0.00000000e+00]
   ...
   [8.01837616e+01 6.02704926e+01 5.28266602e+01]
   [7.92193832e+01 5.94336700e+01 5.23622398e+01]
   [8.59746552e+01 6.52603683e+01 6.01175156e+01]]

  [[3.80000000e+01 0.00000000e+00 0.00000000e+00]
   [3.80000000e+01 0.00000000e+00 0.00000000e+00]
   [3.65714264e+01 0.00000000e+00 0.00000000e+00]
   ...
   [5.09030342e+01 2.24744606e+01 1.89030342e+01]
   [4.70714264e+01 1.86428566e+01 1.58571415e+01]
   [5.17654152e+01 2.31225567e+01 2.03368416e+01]]

  ...

  [[3.44387589e+01 1.28061998e+00 0.00000000e+00]
   [3